In [1]:
!pip install opencv-python
!pip install gdown
!pip install timm

In [2]:
import gdown

google_path = 'https://drive.google.com/uc?id='
file_id = '1AWOO1awvSGHHKbydWJTmeZ0g5f5rV85I'
output_name = 'open.zip'
gdown.download(google_path+file_id,output_name,quiet=False)

Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1AWOO1awvSGHHKbydWJTmeZ0g5f5rV85I 



In [3]:
!unzip open.zip

Archive:  open.zip
replace open/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [5]:
!unzip test.zip
!unzip train.zip

Archive:  test.zip
replace test/20000.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
Archive:  train.zip
replace train/10000.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


### 라이브러리 불러오기

In [22]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time


device = torch.device('cuda')

In [23]:
train_png = sorted(glob('/home/open/train/*.png'))
test_png = sorted(glob('/home/open/test/*.png'))

In [24]:
train_y = pd.read_csv("/home/open/train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [25]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (512, 512))
    return img

In [26]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|██████████| 2154/2154 [01:15<00:00, 28.63it/s]


In [27]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            augmentation = random.randint(0,2)
            if augmentation==1:
                img = img[::-1].copy()
            elif augmentation==2:
                img = img[:,::-1].copy()
        img = transforms.ToTensor()(img)
        if self.mode=='test':
            pass
        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('res2net101_26w_4s', pretrained=True, num_classes=88)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [28]:
batch_size = 8
epochs = 150

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train')
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

### 모델 학습

In [ ]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

model = Network().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 



best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    
    train_f1 = score_function(train_y, train_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-res2net/res2net101_26w_4s-02a759a1.pth" to /root/.cache/torch/hub/checkpoints/res2net101_26w_4s-02a759a1.pth


epoch : 1/150    time : 139s/20720s
TRAIN    loss : 2.06364    f1 : 0.09100
epoch : 2/150    time : 140s/20764s
TRAIN    loss : 1.03610    f1 : 0.14737
epoch : 3/150    time : 141s/20775s
TRAIN    loss : 0.88601    f1 : 0.15170
epoch : 4/150    time : 142s/20702s
TRAIN    loss : 0.81507    f1 : 0.15692
epoch : 5/150    time : 141s/20431s
TRAIN    loss : 0.79141    f1 : 0.16130
epoch : 6/150    time : 142s/20514s
TRAIN    loss : 0.77151    f1 : 0.16496
epoch : 7/150    time : 140s/20026s
TRAIN    loss : 0.73975    f1 : 0.16158
epoch : 8/150    time : 140s/19844s
TRAIN    loss : 0.74000    f1 : 0.17928
epoch : 9/150    time : 140s/19805s
TRAIN    loss : 0.73287    f1 : 0.17237
epoch : 10/150    time : 140s/19644s
TRAIN    loss : 0.73038    f1 : 0.17901
epoch : 11/150    time : 137s/19075s
TRAIN    loss : 0.69862    f1 : 0.18369
epoch : 12/150    time : 139s/19244s
TRAIN    loss : 0.69040    f1 : 0.18137
epoch : 13/150    time : 139s/19029s
TRAIN    loss : 0.68934    f1 : 0.18613
epoch : 

### 추론

In [ ]:
model.eval()
f_pred = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [ ]:
label_decoder = {val:key for key, val in label_unique.items()}
f_result = [label_decoder[result] for result in f_pred]

### 제출물 생성

In [ ]:
submission = pd.read_csv("/home/open/sample_submission.csv")
submission["label"] = f_result
submission

In [ ]:
submission.to_csv("baseline.csv", index = False)